# Potential Talents

The goal of this project is to predict how fit the candidate is based on their available information by ranking them, and then re-rank candidates when a candidate is starred

## Importing Libraries

In [3]:
import time
import numpy as np
import pandas as pd
import shap
import lightgbm as lgm
import pandas_profiling
import sweetviz
import dtale
import warnings, gc
from pycaret.classification import *
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.graph_objects as go
from collections import Counter

import gensim.downloader as api
from gensim.test.utils import get_tmpfile
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
warnings.filterwarnings("ignore")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
temp=dict(layout=go.Layout(font=dict(family="Arial", size=12), 
                           height=500, width=1000))

/var/folders/jz/wqbl3_y10y7f68sltpqpz24m0000gn/T/ipykernel_4137/4017000546.py:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


NameError: name 'nltk' is not defined

## EDA - Exploratory Data Analysis

In [4]:
#the two terms we are looking for as job titles
query1 = 'aspiring human resources'
query2 = 'seeking human resources'

In [89]:
df = pd.read_csv("potential-talents - Aspiring human resources - seeking human resources.csv")
df

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


Check for unique values in each column

In [91]:
for col in df.columns:
    print(f'There is {df[col].nunique()} unique value in {col} column')

There is 104 unique value in id column
There is 52 unique value in job_title column
There is 41 unique value in location column
There is 33 unique value in connection column
There is 0 unique value in fit column


In [92]:
#check for duplicate rows
df_dup = df.drop(['id','fit'], axis = 1)
len(df_dup)-len(df_dup.drop_duplicates())

51

Removing duplicate rows:

In [93]:
df_dup = df_dup.drop_duplicates()                                    
df = pd.concat([df['id'], df_dup], axis = 1).dropna(axis = 0)
print("Shape of non-duplicated dataframe:", df.shape)  

Shape of non-duplicated dataframe: (53, 4)


In [94]:
df

,id,job_title,location,connection
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1
6,7,Student at Humber College and Aspiring Human R...,Kanada,61
7,8,HR Senior Specialist,San Francisco Bay Area,500+
9,10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+
10,11,Student at Chapman University,"Lake Forest, California",2


In [95]:
df = df.reset_index(drop = True)

## Preprocessing

First, we convert every job title to lower string

In [96]:
df['job_title'] = df['job_title'].str.lower()
df

,id,job_title,location,connection
0,1,2019 c.t. bauer college of business graduate (...,"Houston, Texas",85
1,2,native english teacher at epik (english progra...,Kanada,500+
2,3,aspiring human resources professional,"Raleigh-Durham, North Carolina Area",44
3,4,people development coordinator at ryan,"Denton, Texas",500+
4,5,advisory board member at celal bayar university,"İzmir, Türkiye",500+
5,6,aspiring human resources specialist,Greater New York City Area,1
6,7,student at humber college and aspiring human r...,Kanada,61
7,8,hr senior specialist,San Francisco Bay Area,500+
8,10,seeking human resources hris and generalist po...,Greater Philadelphia Area,500+
9,11,student at chapman university,"Lake Forest, California",2


Now we can also have a look at the most frequent words

In [97]:
words_counts = Counter()
for i in range(len(df)):
    for word in df.job_title[i].split(" "):
        words_counts[word] += 1

print('Number of total words', len(words_counts))
words_counts.most_common()

Number of total words 210


[('human', 34),
 ('resources', 29),
 ('at', 22),
 ('and', 13),
 ('aspiring', 13),
 ('seeking', 10),
 ('|', 10),
 ('professional', 9),
 ('in', 6),
 ('university', 6),
 ('student', 6),
 ('of', 5),
 ('business', 5),
 ('generalist', 5),
 ('manager', 5),
 ('specialist', 4),
 ('&', 4),
 ('management', 4),
 ('an', 4),
 ('', 4),
 ('director', 3),
 ('position', 3),
 ('for', 3),
 ('college', 2),
 ('coordinator', 2),
 ('hr', 2),
 ('senior', 2),
 ('the', 2),
 ('internship', 2),
 ('opportunities', 2),
 ('resources,', 2),
 ('staffing', 2),
 ('north', 2),
 ('manager,', 2),
 ('a', 2),
 ('resources.', 2),
 ('major', 2),
 ('to', 2),
 ('information', 2),
 ('systems', 2),
 ('leader', 2),
 ('-', 2),
 ('2019', 1),
 ('c.t.', 1),
 ('bauer', 1),
 ('graduate', 1),
 ('(magna', 1),
 ('cum', 1),
 ('laude)', 1),
 ('native', 1),
 ('english', 1),
 ('teacher', 1),
 ('epik', 1),
 ('(english', 1),
 ('program', 1),
 ('korea)', 1),
 ('people', 1),
 ('development', 1),
 ('ryan', 1),
 ('advisory', 1),
 ('board', 1),
 ('member', 1),
 ('celal', 1),
 ('bayar', 1),
 ('humber', 1),
 ('hris', 1),
 ('positions', 1),
 ('chapman', 1),
 ('svp,', 1),
 ('chro,', 1),
 ('marketing', 1),
 ('communications,', 1),
 ('csr', 1),
 ('officer', 1),
 ('engie', 1),
 ('houston', 1),
 ('woodlands', 1),
 ('energy', 1),
 ('gphr', 1),
 ('sphr', 1),
 ('intercontinental', 1),
 ('buckhead', 1),
 ('atlanta', 1),
 ('experienced', 1),
 ('retail', 1),
 ('recruiting', 1),
 ('luxottica', 1),
 ('america,', 1),
 ('groupe', 1),
 ('beneteau', 1),
 ('retired', 1),
 ('army', 1),
 ('national', 1),
 ('guard', 1),
 ('recruiter,', 1),
 ('office', 1),
 ('scottmadden,', 1),
 ('inc.', 1),
 ('nortia', 1),
 ('is', 1),
 ('payroll', 1),
 ('administrative', 1),
 ('professionals!!', 1),
 ('(408)', 1),
 ('709-2621', 1),
 ('passionate', 1),
 ('about', 1),
 ('helping', 1),
 ('create', 1),
 ('inclusive', 1),
 ('engaging', 1),
 ('work', 1),
 ('environment', 1),
 ('resources|\nconflict', 1),
 ('management|\npolicies', 1),
 ('procedures|talent', 1),
 ('management|benefits', 1),
 ('compensation', 1),
 ("schwan's", 1),
 ('liberal', 1),
 ('arts', 1),
 ('major.', 1),
 ('analyst.', 1),
 ('junior', 1),
 ('mes', 1),
 ('engineer|', 1),
 ('partner', 1),
 ('heil', 1),
 ('environmental', 1),
 ('energetic', 1),
 ('team-focused', 1),
 ('endemol', 1),
 ('shine', 1),
 ('america', 1),
 ('world', 1),
 ('gis', 1),
 ('software', 1),
 ('rrp', 1),
 ('brand', 1),
 ('portfolio', 1),
 ('executive', 1),
 ('jti', 1),
 ('(japan', 1),
 ('tobacco', 1),
 ('international)', 1),
 ('programmer', 1),
 ('with', 1),
 ('love', 1),
 ('data', 1),
 ('organization.', 1),
 ('bachelor', 1),
 ('science', 1),
 ('biology', 1),
 ('from', 1),
 ('victoria', 1),
 ('wellington', 1),
 ('ey', 1),
 ('undergraduate', 1),
 ('research', 1),
 ('assistant', 1),
 ('styczynski', 1),
 ('lab', 1),
 ('lead', 1),
 ('official', 1),
 ('western', 1),
 ('illinois', 1),
 ('employment', 1),
 ('within', 1),
 ('customer', 1),
 ('service', 1),
 ('or', 1),
 ('patient', 1),
 ('care', 1),
 ('admissions', 1),
 ('representative', 1),
 ('community', 1),
 ('medical', 1),
 ('center', 1),
 ('long', 1),
 ('beach', 1),
 ('opportunities.', 1),
 ('open', 1),
 ('travel', 1),
 ('relocation.', 1),
 ('westfield', 1),
 ('state', 1),
 ('indiana', 1),
 ('kokomo', 1),
 ('\nretail', 1),
 ('delphi', 1),
 ('hardware', 1),
 ('paint', 1),
 ('graduating', 1),
 ('may', 1),
 ('2020', 1),
 ('entry-level', 1),
 ('st.', 1),
 ('louis', 1),
 ('loparex', 1),
 ('intelligence', 1),
 ('analytics', 1),
 ('travelers', 1),
 ('always', 1),
 ('set', 1),
 ('them', 1),
 ('up', 1),
 ('success', 1),
 ('administration', 1),
 ('excellence', 1),
 ('logging', 1)]

Next we are going to remove stop words and apply lemmatization, which is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meanings to one word

In [98]:
def cleaning(df, col):
    
    stop_words = set(stopwords.words('english'))
    for i in range (len(df)):
        word_tokens = word_tokenize(df[col][i])
        tokens_without_sw = [w for w in word_tokens if w not in stop_words]
        lemmatized_sentence = []
        for word in tokens_without_sw:
            lemmatized_sentence.append(WordNetLemmatizer().lemmatize(word))
        df[col][i] = TreebankWordDetokenizer().detokenize(lemmatized_sentence) 

In [99]:
print('Job title before removing stopwprds:\n', df.job_title.head(1))
print("-" * 100)
df.head()

Job title before removing stopwprds:
 0    2019 c.t. bauer college of business graduate (...
Name: job_title, dtype: object
----------------------------------------------------------------------------------------------------


,id,job_title,location,connection
0,1,2019 c.t. bauer college of business graduate (...,"Houston, Texas",85
1,2,native english teacher at epik (english progra...,Kanada,500+
2,3,aspiring human resources professional,"Raleigh-Durham, North Carolina Area",44
3,4,people development coordinator at ryan,"Denton, Texas",500+
4,5,advisory board member at celal bayar university,"İzmir, Türkiye",500+


In [100]:
# Remove stop words and lemmatize words in cleaned job title
cleaning(df, 'job_title')

print('Job title after removing stopwprds:\n', df.job_title.head(1))
print("-" * 100)
df.head()

Job title after removing stopwprds:
 0    2019 c.t . bauer college business graduate (ma...
Name: job_title, dtype: object
----------------------------------------------------------------------------------------------------


,id,job_title,location,connection
0,1,2019 c.t . bauer college business graduate (ma...,"Houston, Texas",85
1,2,native english teacher epik (english program k...,Kanada,500+
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44
3,4,people development coordinator ryan,"Denton, Texas",500+
4,5,advisory board member celal bayar university,"İzmir, Türkiye",500+


As a last step in the preprocessing phase, we are going to standardaize all the acronyms.

In [101]:
df = df.replace({'job_title' : { 'chro' : 'chief human resources officer', 'svp' : 'senior vice president'
        ,'gphr' : 'global professional in human resources','hris' : 'human resources management system'
        , 'csr' : 'corporate social responsibility', 'sphr' : 'strategic and policy-making certification'
        , 'hr' : 'human resources', "[\'!#)$%&(*+-./:;<=>?@[\]^_`{|}~\n]" : "", r'[0-9]' : '', 'epik' : 'tech'
        , 'styczynski lab' : 'tech', 'gi' : 'tech', "schwan's" : 'tech', 'ryan' : 'not tech'
        , 'engine' : 'not tech', 'buckhead atlanta' : 'not tech', 'loparex' : 'not tech', 'delphi Hardware' : 'not tech'
        , "jti" : 'not tech', 'traveler' : 'not tech', 'luxottica' : 'not tech'
        , 'beneteau' : 'not tech', 'scottMadden' : 'not tech', 'ey' : 'not tech', 'endemol' : 'not tech'
        , 'nortia staffing' : 'not tech', 'heil environmental' : 'not tech', 'excellence logging' : 'not tech'}}
        , regex=True)

## Word Embedding Techniques

### 1. TF-IDF

In [102]:
# Convert job_title column into a list
job_title_list = list(df['job_title'])                    

# Vectorize job_title_list
vectorizer = TfidfVectorizer().fit(job_title_list)                    
X = vectorizer.transform(job_title_list)   

# Get feature names in all the documents
feature_names = vectorizer.get_feature_names()                  
print("Number of unique features: ", len(feature_names)) 
print("First 5 features: ", feature_names[:5]) 

# Convert job titles into arrays
tfidf_vector = X.toarray()                                  
print("Shape of Tfidf vector: ", tfidf_vector.shape)  

Number of unique features:  167
First 5 features:  ['administration', 'administrative', 'admission', 'advisory', 'always']
Shape of Tfidf vector:  (53, 167)


In [103]:
def similarity(phrase, column_name):
    # Convert search phrase into a vector
    X = vectorizer.transform(phrase)                    
    X_vector = X.toarray()
    print("Shape of search phrase vector:", X_vector.shape)
    
    # Calcualte Tfidf cosine similarity and add it to pt dataframe
    sim_score_list = []
    for x in range (len(df)):
        #Note that spatial.distance.cosine computes the distance, and not the similarity. So, you must subtract the value from 1 to get the similarity.
        sim_score_list.append(1 - cosine(tfidf_vector[x], X_vector.reshape((167,))))

    df[column_name] = sim_score_list

In [104]:
# Searched phrase
phrases_1 = pd.DataFrame({'phrase' : ['aspiring human resources']})
cleaning(phrases_1, 'phrase')
phrases_2 = pd.DataFrame({'phrase' : ['seeking human resources']})
cleaning(phrases_2, 'phrase')

In [105]:
similarity([phrases_1.phrase[0]], 'TF-IDF_score_aspiring_human_resources')

similarity([phrases_2.phrase[0]], 'TF-IDF_score_seeking_human_resources')

Shape of search phrase vector: (1, 167)
Shape of search phrase vector: (1, 167)


In [106]:
df.sort_values(by ='TF-IDF_score_aspiring_human_resources', ascending = False).head()

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366


In [107]:
df.sort_values(by ='TF-IDF_score_seeking_human_resources', ascending = False).head()

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298


In [108]:
df['TF-IDF_fit'] = df[["TF-IDF_score_seeking_human_resources", "TF-IDF_score_aspiring_human_resources"]].max(axis=1)

In [147]:
df.sort_values(by ='TF-IDF_fit', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources,BERT_fit,FT_score_aspiring_human_resources,FT_score_seeking_human_resources,FT_fit,GloVe_score_aspiring_human_resources,GloVe_score_seeking_human_resources,GloVe_fit,starred_score
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076,0.958512,1.000000
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076,0.958512,0.748700
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737,0.901517,0.901517,0.901517,0.850613,0.959728,0.959728,0.869983,0.967423,0.967423,0.901517
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.762933,0.762933,0.762933,0.969928,0.884207,0.969928,0.969614,0.896009,0.969614,0.762933
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.883154,0.883154,0.883154,0.877444,0.977981,0.977981,0.894528,0.981906,0.981906,0.883154
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.689070,0.689070,0.689070,0.938275,0.931645,0.938275,0.934672,0.942569,0.942569,0.689070
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366,0.468366,0.783973,0.718755,0.783973,0.599141,0.599141,0.599141,0.866789,0.845641,0.866789,0.881137,0.874048,0.881137,0.599141
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203,0.721716,0.721716,0.721716,0.859765,0.919272,0.919272,0.871824,0.949446,0.949446,0.721716
20,72,business management major aspiring human resou...,"Monroe, Louisiana Area",5,0.439855,0.181739,0.439855,0.760062,0.668805,0.760062,0.548536,0.548536,0.548536,0.812208,0.736214,0.812208,0.812279,0.840596,0.840596,0.548536
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.722961,0.722961,0.722961,0.931923,0.911936,0.931923,0.933199,0.916529,0.933199,0.722961


In the TF-IDF model, we can see that none of the titles reached the cosine similarity of 1 since all candidates have different job titles from our benchmarks. However, those closer to the queries received a score higher than zero. So we could cut off from the point where the score is zero for this model.

### 2. Word2Vec

In [26]:
GoogleNews_model = api.load('word2vec-google-news-300')

In [110]:
def doc_token_vectors(sentence, model, sentence_vector_list, vector_dimensions):
    word_tokens = word_tokenize(sentence)
    filtered_words = [w for w in word_tokens if w in model]
    for j in range(len(word_tokens)):
        if word_tokens[j] in filtered_words:
            token_vector = model[word_tokens[j]]
        else:
            token_vector = np.zeros(vector_dimensions)
        sentence_vector_list.append(token_vector)
    
    return sentence_vector_list

In [111]:
def model_fitt_score(df, col, model, vector_dimensions, phrase, fitt_col_name):
    # Vectorize job title using the model
    model_vectors = []
    for i in range(len(df)):
        model_sentence_vector = []
        doc_token_vectors(df[col][i], model, model_sentence_vector, vector_dimensions)
        model_vectors.append(model_sentence_vector)

    print('model vectors shape', np.shape(model_vectors))
    # Vectorize searched phrase using the model
    model_search_phrase_vector = []
    doc_token_vectors(phrase, model, model_search_phrase_vector, vector_dimensions)
    print('model search phrase vector shape', np.shape(model_search_phrase_vector))

    # Calculate cosine similarity between searched phrase and job title
    model_similarity =[]
    for i in range(len(df)):
        sim_score = 1 - cosine(np.mean(model_vectors[i], axis = 0), np.mean(model_search_phrase_vector, axis =0))
        model_similarity.append(sim_score)

    # Add model similarity score to the pt dataframe
    df[fitt_col_name] = model_similarity
    # return df

In [112]:
model_fitt_score(df, 'job_title', GoogleNews_model, 300, phrases_1.phrase[0], 'Word2Vec_score_aspiring_human_resources')

model vectors shape (53,)
model search phrase vector shape (3, 300)


In [113]:
model_fitt_score(df, 'job_title', GoogleNews_model, 300, phrases_2.phrase[0], 'Word2Vec_score_seeking_human_resources')

model vectors shape (53,)
model search phrase vector shape (3, 300)


In [114]:
df.sort_values(by ='Word2Vec_score_aspiring_human_resources', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299
6,7,student humber college aspiring human resource...,Kanada,61,0.395109,0.163251,0.395109,0.829068,0.632909
30,82,aspiring human resource professional energeti...,"Austin, Texas Area",174,0.379605,0.156845,0.379605,0.827266,0.639151
48,100,aspiring human resource manager graduating ma...,"Cape Girardeau, Missouri",103,0.378556,0.387665,0.387665,0.791290,0.771333
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366,0.468366,0.783973,0.718755
14,66,experienced retail manager aspiring human reso...,"Austin, Texas Area",57,0.410921,0.169784,0.410921,0.779507,0.625699


In [115]:
df['Word2Vec_fit'] = df[["Word2Vec_score_aspiring_human_resources", "Word2Vec_score_seeking_human_resources"]].max(axis=1)

In [116]:
df.sort_values(by ='Word2Vec_fit', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203
6,7,student humber college aspiring human resource...,Kanada,61,0.395109,0.163251,0.395109,0.829068,0.632909,0.829068
30,82,aspiring human resource professional energeti...,"Austin, Texas Area",174,0.379605,0.156845,0.379605,0.827266,0.639151,0.827266


For the Word2Vec model, we reached higher cosine similarity scores and the first job titles for the rank seem to make sense. However, it might be harder to set a threshold to cut off those candidates not well suited to the position since it depends on the interpretation to define it. In this case, it seems to make sense to filter all scores above 0.72 of similarity.

### 3. BERT

In [43]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 28.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 5.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=89aaa53bb8285f1af205066f3e9a85cab2642d35a2c9a4cd20d9be056ebbe9bf
  Stored in directory: /Users/vitorhugoteixeiradeluccafilho/Library/Caches/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_tr

In [117]:
#https://www.sbert.net/docs/pretrained_models.html
from sentence_transformers import SentenceTransformer
# Build BERT_base model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [118]:
# Convert job titles into BERT embedded vectors
bert_job_title_embeddings = bert_model.encode(list(df.job_title))
bert_job_title_embeddings.shape

(53, 384)

In [119]:
# Convert search phrase into a BERT embedded vector
bert_search_phrase_embedding = bert_model.encode(phrases_1.phrase[0])
bert_search_phrase_embedding.shape

(384,)

In [125]:
# Calculate cosine similarity between job title and search phrase vectors
bert_cosine_similarity = []
for i in range(len(df)):
    cos_sim = 1 - cosine(bert_job_title_embeddings[i], bert_search_phrase_embedding)
    bert_cosine_similarity.append(cos_sim)
    
# Add BERT_cosine_similarity column in the pt dataframe
df['BERT_score_aspiring_human_resources'] = bert_cosine_similarity

In [126]:
# Convert search phrase into a BERT embedded vector
bert_search_phrase_embedding = bert_model.encode(phrases_2.phrase[0])
bert_search_phrase_embedding.shape

# Calculate cosine similarity between job title and search phrase vectors
bert_cosine_similarity = []
for i in range(len(df)):
    #Note that spatial.distance.cosine computes the distance, and not the similarity. So, you must subtract the value from 1 to get the similarity.
    cos_sim = 1 - cosine(bert_job_title_embeddings[i], bert_search_phrase_embedding)
    bert_cosine_similarity.append(cos_sim)
    
# Add BERT_cosine_similarity column in the pt dataframe
df['BERT_score_seeking_human_resources'] = bert_cosine_similarity

In [127]:
df.sort_values(by ='BERT_score_seeking_human_resources', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_model_score_aspiring_human_resources,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737,0.798970,0.901517,0.901517
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.807473,0.883154,0.883154
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.917099,0.762933,0.762933
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.793775,0.722961,0.722961
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203,0.686404,0.721716,0.721716
15,67,human resource staffing recruiting professional,"Jackson, Mississippi Area",500+,0.198971,0.187895,0.198971,0.706574,0.671721,0.706574,0.718870,0.719885,0.719885
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.762450,0.689070,0.689070
6,7,student humber college aspiring human resource...,Kanada,61,0.395109,0.163251,0.395109,0.829068,0.632909,0.829068,0.749215,0.640940,0.640940


In [128]:
bert_job_title_embeddings

array([[-0.06890015,  0.02933308,  0.00048102, ..., -0.11115385,
        -0.00875637, -0.0403384 ],
       [-0.0265617 , -0.0255274 ,  0.08100629, ..., -0.00013312,
        -0.02250874,  0.03101562],
       [-0.03940797,  0.06459111, -0.00836966, ..., -0.06119971,
        -0.03338577,  0.03041076],
       ...,
       [ 0.03454832, -0.0315544 , -0.05728424, ..., -0.05954725,
         0.03684887, -0.08669347],
       [-0.01991805,  0.03471512, -0.01384534, ...,  0.0363196 ,
         0.00378118, -0.00329514],
       [ 0.03702419, -0.05630671,  0.00691669, ..., -0.09644348,
         0.04128828,  0.00083805]], dtype=float32)

In [129]:
df['BERT_fit'] = df[["BERT_score_aspiring_human_resources", "BERT_score_seeking_human_resources"]].max(axis=1)
df.sort_values(by ='BERT_fit', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_model_score_aspiring_human_resources,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources,BERT_fit
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737,0.798970,0.901517,0.901517,0.901517
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.807473,0.883154,0.883154,0.883154
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.917099,0.762933,0.762933,0.762933
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.793775,0.722961,0.722961,0.722961
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203,0.686404,0.721716,0.721716,0.721716
15,67,human resource staffing recruiting professional,"Jackson, Mississippi Area",500+,0.198971,0.187895,0.198971,0.706574,0.671721,0.706574,0.718870,0.719885,0.719885,0.719885
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.762450,0.689070,0.689070,0.689070
6,7,student humber college aspiring human resource...,Kanada,61,0.395109,0.163251,0.395109,0.829068,0.632909,0.829068,0.749215,0.640940,0.640940,0.640940


This model worked in a similar way to Word2Vec in getting a high score and not assigning zero. But in this case, Bert gave lower scores to those job titles including the words specialist and professional than the prior, which might not make sense for this problem.

### 4. FastText

In [45]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')

 (42.28%) [=====================>                             ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]====================>                              ]=

 (100.00%) [==================================================>]================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=

'cc.en.300.bin'

In [46]:
ft = fasttext.load_model('cc.en.300.bin')

In [130]:
model_fitt_score(df, 'job_title', ft, 300, phrases_1.phrase[0], 'FT_score_aspiring_human_resources')

model vectors shape (53,)
model search phrase vector shape (3, 300)


In [131]:
model_fitt_score(df, 'job_title', ft, 300, phrases_2.phrase[0], 'FT_score_seeking_human_resources')

model vectors shape (53,)
model search phrase vector shape (3, 300)


In [132]:
df.sort_values(by ='FT_score_aspiring_human_resources', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_model_score_aspiring_human_resources,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources,BERT_fit,FT_score_aspiring_human_resources,FT_score_seeking_human_resources
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.917099,0.762933,0.762933,0.762933,0.969928,0.884207
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.762450,0.689070,0.689070,0.689070,0.938275,0.931645
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.793775,0.722961,0.722961,0.722961,0.931923,0.911936
30,82,aspiring human resource professional energeti...,"Austin, Texas Area",174,0.379605,0.156845,0.379605,0.827266,0.639151,0.827266,0.566655,0.470699,0.470699,0.470699,0.885884,0.804348
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.807473,0.883154,0.883154,0.883154,0.877444,0.977981
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366,0.468366,0.783973,0.718755,0.783973,0.702917,0.599141,0.599141,0.599141,0.866789,0.845641
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203,0.686404,0.721716,0.721716,0.721716,0.859765,0.919272
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737,0.798970,0.901517,0.901517,0.901517,0.850613,0.959728


In [133]:
df.sort_values(by ='FT_score_seeking_human_resources', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_model_score_aspiring_human_resources,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources,BERT_fit,FT_score_aspiring_human_resources,FT_score_seeking_human_resources
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.807473,0.883154,0.883154,0.883154,0.877444,0.977981
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737,0.798970,0.901517,0.901517,0.901517,0.850613,0.959728
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.762450,0.689070,0.689070,0.689070,0.938275,0.931645
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203,0.686404,0.721716,0.721716,0.721716,0.859765,0.919272
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.793775,0.722961,0.722961,0.722961,0.931923,0.911936
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.917099,0.762933,0.762933,0.762933,0.969928,0.884207
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366,0.468366,0.783973,0.718755,0.783973,0.702917,0.599141,0.599141,0.599141,0.866789,0.845641
15,67,human resource staffing recruiting professional,"Jackson, Mississippi Area",500+,0.198971,0.187895,0.198971,0.706574,0.671721,0.706574,0.718870,0.719885,0.719885,0.719885,0.810696,0.837362


In [134]:
df['FT_fit'] = df[["FT_score_aspiring_human_resources", "FT_score_seeking_human_resources"]].max(axis=1)
df.sort_values(by ='FT_fit', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_model_score_aspiring_human_resources,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources,BERT_fit,FT_score_aspiring_human_resources,FT_score_seeking_human_resources,FT_fit
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.807473,0.883154,0.883154,0.883154,0.877444,0.977981,0.977981
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.917099,0.762933,0.762933,0.762933,0.969928,0.884207,0.969928
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737,0.798970,0.901517,0.901517,0.901517,0.850613,0.959728,0.959728
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.762450,0.689070,0.689070,0.689070,0.938275,0.931645,0.938275
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.793775,0.722961,0.722961,0.722961,0.931923,0.911936,0.931923
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203,0.686404,0.721716,0.721716,0.721716,0.859765,0.919272,0.919272
30,82,aspiring human resource professional energeti...,"Austin, Texas Area",174,0.379605,0.156845,0.379605,0.827266,0.639151,0.827266,0.566655,0.470699,0.470699,0.470699,0.885884,0.804348,0.885884
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366,0.468366,0.783973,0.718755,0.783973,0.702917,0.599141,0.599141,0.599141,0.866789,0.845641,0.866789


Fasttext presented results very similar to Word2Vec, just with a higher cosine similarity score.

### 5. GloVe

In [60]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2023-05-11 19:40:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-11 19:40:39--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-11 19:40:40--  https://downloads.cs.stanford.edu

In [61]:
from gensim.test.utils import get_tmpfile

# Create temp file and save converted embedding into it
target_file = get_tmpfile('word2vec.6B.50d.txt')
glove2word2vec('glove.6B.50d.txt', target_file) 

# Load the converted embedding into memory
glove_model = KeyedVectors.load_word2vec_format(target_file)

# Save as binary data
glove_model.save_word2vec_format('word2vec.6B.50d.bin.gz', binary=True)

In [135]:
model_fitt_score(df, 'job_title', glove_model, 50, phrases_1.phrase[0], 'GloVe_score_aspiring_human_resources')
model_fitt_score(df, 'job_title', glove_model, 50, phrases_2.phrase[0], 'GloVe_score_seeking_human_resources')

model vectors shape (53,)
model search phrase vector shape (3, 50)
model vectors shape (53,)
model search phrase vector shape (3, 50)


In [136]:
df.sort_values(by ='GloVe_score_aspiring_human_resources', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_model_score_aspiring_human_resources,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources,BERT_fit,FT_score_aspiring_human_resources,FT_score_seeking_human_resources,FT_fit,GloVe_score_aspiring_human_resources,GloVe_score_seeking_human_resources
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.917099,0.762933,0.762933,0.762933,0.969928,0.884207,0.969928,0.969614,0.896009
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.943130,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.762450,0.689070,0.689070,0.689070,0.938275,0.931645,0.938275,0.934672,0.942569
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.793775,0.722961,0.722961,0.722961,0.931923,0.911936,0.931923,0.933199,0.916529
24,76,aspiring human resource professional passiona...,"New York, New York",212,0.259969,0.107414,0.259969,0.741262,0.625898,0.741262,0.635659,0.596428,0.596428,0.596428,0.777573,0.758395,0.777573,0.917865,0.889851
30,82,aspiring human resource professional energeti...,"Austin, Texas Area",174,0.379605,0.156845,0.379605,0.827266,0.639151,0.827266,0.566655,0.470699,0.470699,0.470699,0.885884,0.804348,0.885884,0.895589,0.826986
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.807473,0.883154,0.883154,0.883154,0.877444,0.977981,0.977981,0.894528,0.981906
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366,0.468366,0.783973,0.718755,0.783973,0.702917,0.599141,0.599141,0.599141,0.866789,0.845641,0.866789,0.881137,0.874048
49,101,human resource generalist not tech,"Raleigh-Durham, North Carolina Area",500+,0.268911,0.253942,0.268911,0.738248,0.731151,0.738248,0.626183,0.581035,0.581035,0.581035,0.728668,0.725355,0.728668,0.878405,0.905816


In [139]:
df['GloVe_fit'] = df[["GloVe_score_aspiring_human_resources", "GloVe_score_seeking_human_resources"]].max(axis=1)
df.sort_values(by ='GloVe_fit', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,BERT_score_seeking_human_resources,BERT_score_aspiring_human_resources,BERT_fit,FT_score_aspiring_human_resources,FT_score_seeking_human_resources,FT_fit,GloVe_score_aspiring_human_resources,GloVe_score_seeking_human_resources,GloVe_fit
13,28,seeking human resource opportunity,"Chicago, Illinois",390,0.279908,0.677450,0.677450,0.756767,0.904507,0.904507,0.883154,0.883154,0.883154,0.877444,0.977981,0.977981,0.894528,0.981906,0.981906
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,0.762933,0.762933,0.762933,0.969928,0.884207,0.969928,0.969614,0.896009,0.969614
47,99,seeking human resource position,"Las Vegas, Nevada Area",48,0.289466,0.700583,0.700583,0.728513,0.918737,0.918737,0.901517,0.901517,0.901517,0.850613,0.959728,0.959728,0.869983,0.967423,0.967423
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076,0.958512
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,0.748700,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076,0.958512
8,10,seeking human resource human resources managem...,Greater Philadelphia Area,500+,0.237450,0.464298,0.464298,0.729699,0.845203,0.845203,0.721716,0.721716,0.721716,0.859765,0.919272,0.919272,0.871824,0.949446,0.949446
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,0.689070,0.689070,0.689070,0.938275,0.931645,0.938275,0.934672,0.942569,0.942569
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,0.722961,0.722961,0.722961,0.931923,0.911936,0.931923,0.933199,0.916529,0.933199
24,76,aspiring human resource professional passiona...,"New York, New York",212,0.259969,0.107414,0.259969,0.741262,0.625898,0.741262,0.596428,0.596428,0.596428,0.777573,0.758395,0.777573,0.917865,0.889851,0.917865
16,68,human resource specialist not tech,Greater New York City Area,500+,0.262914,0.248279,0.262914,0.693958,0.728842,0.728842,0.622262,0.622262,0.622262,0.715083,0.717646,0.717646,0.866703,0.914137,0.914137


GloVe also are aligned with the problem but considered the job title of directors and SVP better ranked than other aspiring candidates. Moreover, the lowest score was above 0.4, even though the job title doesn't have any relation to the queries.

### Dynamic Re-ranking

To create a dynamic re-ranking we are going to use TF-IDF because with this model we can separete candidates who should not be in this list in a easier way just excluding those with cosine score equals to zero, and also since the model worked well in ranking the job titles using the queries.

In [148]:
star_candidate = input("Do you want to star any candidates? Enter 'Yes' or 'No': ")

starred = []
if star_candidate.lower() == 'yes':
    starred = [int(item) for item in input("Enter the id of the candidate you want to star: ").split()]

Do you want to star any candidates? Enter 'Yes' or 'No': yes
Enter the id of the candidate you want to star: 3


In [166]:
starred_candidate = df.loc[df['id'] == 3]['job_title'].item()

In [168]:
similarity([starred_candidate], 'TF-IDF_starred_candidate')

Shape of search phrase vector: (1, 167)


In [169]:
df.sort_values(by ='TF-IDF_starred_candidate', ascending = False)

,id,job_title,location,connection,TF-IDF_score_aspiring_human_resources,TF-IDF_score_seeking_human_resources,TF-IDF_fit,Word2Vec_score_aspiring_human_resources,Word2Vec_score_seeking_human_resources,Word2Vec_fit,...,BERT_score_aspiring_human_resources,BERT_fit,FT_score_aspiring_human_resources,FT_score_seeking_human_resources,FT_fit,GloVe_score_aspiring_human_resources,GloVe_score_seeking_human_resources,GloVe_fit,starred_score,TF-IDF_starred_candidate
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",44,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,...,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076,0.958512,1.000000,1.000000
45,97,aspiring human resource professional,"Kokomo, Indiana Area",71,0.781184,0.322769,0.781184,0.950395,0.723763,0.950395,...,0.748700,0.748700,0.984363,0.877064,0.984363,0.958512,0.866076,0.958512,0.781184,1.000000
22,74,human resource professional,Greater Boston Area,16,0.421758,0.398280,0.421758,0.874494,0.799299,0.874494,...,0.722961,0.722961,0.931923,0.911936,0.931923,0.933199,0.916529,0.933199,0.421758,0.810405
5,6,aspiring human resource specialist,Greater New York City Area,1,0.679890,0.280916,0.679890,0.912262,0.759907,0.912262,...,0.762933,0.762933,0.969928,0.884207,0.969928,0.969614,0.896009,0.969614,0.679890,0.531119
14,66,experienced retail manager aspiring human reso...,"Austin, Texas Area",57,0.410921,0.169784,0.410921,0.779507,0.625699,0.779507,...,0.576206,0.576206,0.833309,0.752087,0.833309,0.833745,0.796197,0.833745,0.410921,0.526024
30,82,aspiring human resource professional energeti...,"Austin, Texas Area",174,0.379605,0.156845,0.379605,0.827266,0.639151,0.827266,...,0.470699,0.470699,0.885884,0.804348,0.885884,0.895589,0.826986,0.895589,0.379605,0.485936
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",7,0.618600,0.633486,0.633486,0.875945,0.842598,0.875945,...,0.689070,0.689070,0.938275,0.931645,0.938275,0.934672,0.942569,0.942569,0.633486,0.483240
15,67,human resource staffing recruiting professional,"Jackson, Mississippi Area",500+,0.198971,0.187895,0.198971,0.706574,0.671721,0.706574,...,0.719885,0.719885,0.810696,0.837362,0.837362,0.848083,0.847649,0.848083,0.198971,0.382321
12,27,aspiring human resource management student see...,"Houston, Texas Area",500+,0.442293,0.468366,0.468366,0.783973,0.718755,0.783973,...,0.599141,0.599141,0.866789,0.845641,0.866789,0.881137,0.874048,0.881137,0.468366,0.345512
20,72,business management major aspiring human resou...,"Monroe, Louisiana Area",5,0.439855,0.181739,0.439855,0.760062,0.668805,0.760062,...,0.548536,0.548536,0.812208,0.736214,0.812208,0.812279,0.840596,0.840596,0.439855,0.343607


### Conclusion

In re-ranking the dataset we placed those candidates starred with 1 cosine similarity and the new rank did a good job in selecting the best-suited candidates for the query, as well as for filtering those that should not be in this list.
In order to automate this procedure to prevent human bias I also would evaluate other sections of the LinkedIn profiles such as "About" and professional experience to enrich the model and consider different keywords.

In re-ranking the dataset we placed those candidates starred with 1 cosine similarity and the new rank did a good job in selecting the best-suited candidates for the query, as well as for filtering those that should not be in this list.
In order to automate this procedure to prevent human bias I also would evaluate other sections of the LinkedIn profiles such as "About" and professional experience to enrich the model and consider different keywords.

# References

https://www.kdnuggets.com/2021/11/guide-word-embedding-techniques-nlp.html

https://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795

https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

https://towardsdatascience.com/a-guide-to-encoding-text-in-python-ef783e50f09e

https://towardsdatascience.com/string-matching-with-fuzzywuzzy-e982c61f8a84

https://www.geeksforgeeks.org/fuzzywuzzy-python-library/

https://www.analyticsvidhya.com/blog/2021/06/fuzzywuzzy-python-library-interesting-tool-for-nlp-and-text-analytics/ 

https://datascience.stackexchange.com/questions/63325/cosine-similarity-vs-the-levenshtein-distance#:~:text=As%20mentioned%20in%20other%20answers,distance%20between%20sequences%20of%20characters

http://jalammar.github.io/illustrated-bert/

https://fasttext.cc/docs/en/crawl-vectors.html
